In [33]:
# Import all necessary library and function

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import requests
import pandas as pd
import io
#!pip install nltk
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# Defining the function to remove unnecesary symbols and empty arrays

def remove(text):
  pattern = r'[“”‘’:;"?_\',.()\–\[\]]'
  sub_text = re.sub(pattern,'', text)
  pattern = r'[\-\–]'
  sub_text = re.sub(pattern,' ', sub_text)
  token_text = word_tokenize(sub_text)
  words = [word for word in token_text if word]
  return ' '.join(words)

In [35]:
# Cleaning the text by iterating through the line

def get_list(data):
  list_arr = []
  text_to_line = sent_tokenize(data.lower())
  for text in text_to_line:
    clean_line = remove(text)
    list_arr.append(clean_line)
  return list_arr

In [36]:
# Creating the text array of all files

def text_list(story_data):
  lists = []
  for _, row in story_data.iterrows():
    text_arr = get_list(row[-4])
    lists.append(text_arr)
  return lists

In [37]:
# Defining the Token of each text

def tokenizing(flat_text,text):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(flat_text)
  word_index = tokenizer.word_index
  tokenid = []
  for i in text:
    tokens = tokenizer.texts_to_sequences(i)
    tokenid.append(tokens)
  return tokenid

In [38]:
# Defining the TfidfVectorizer of each text

def vecTfid(flat_text,text):
  vec = TfidfVectorizer()
  vec.fit(flat_text)
  vecarr = []
  for i in text:
    transform = vec.transform(i).toarray()
    vecarr.append(transform)
  return vecarr

In [85]:
def add_values(story_dict, key, values, id):
    # Define the fiction and chapter id
    fic_id, chap_id = id

    # Set the dictionary
    story_dict.setdefault(key, {}).setdefault(fic_id, {}).setdefault(chap_id, [])

    # Adding the lines into each dictionary keys
    for line in values:
        if line not in story_dict[key][fic_id][chap_id]:
            story_dict[key][fic_id][chap_id].append(line)

In [75]:
# Using the flag system to count copy-pasted and high similarity line

def count_flag(token1, token2, tf1, tf2, story_dict, text1, text2, title):
  # Defining all necessary flag variables
  plag_tf_token = 0
  di = len(tf1)
  token_cos = []
  cp = 0
  hs = 0

  # Getting the similarity on Tokenized text and Vectorized text
  tf_cos = cosine_similarity(tf1,tf2)
  for i in range(len(token1)):
    cos_line= []
    for j in range(len(token2)):
      length = len(list(set(token1[i]+token2[j])))
      cos = len(set(token1[i])&set(token2[j]))/length
      cos_line.append(cos)
    token_cos.append(cos_line)

  """
  Both Tokenized and Vectorized has their own strength and uses
  Similarity threshold (0.9999, 0.7, and 0.35) may be changed
  Flag system works by:
    1. Flag each line that are copy-pasted or high similarity with value 1, final result is the total line
    2. Line with less than 0.9999 similarity uses several threshold (0.7 and 0.3)
          - Both threshold gave different flag value (for tf) (plag_tf_token)
          - Final result of plag_tf_token is the summation of flag tf and token if total > 1
  """
  for i in range(di):
    tf = 0
    token = 0
    #print(tf_cos[i])
    #print(token_cos[i])
    # For high similarity checking
    if any(vals >= 0.9999 for vals in tf_cos[i]):
      tryis = [index for index, vals in enumerate(tf_cos[i]) if vals >= 0.9999]
      add_values(story_dict, text1[i],[text2[i] for i in tryis], title)
      hs+=1
    elif any(0.35 < vals < 0.9999 for vals in tf_cos[i]):
      tryis = [index for index, vals in enumerate(tf_cos[i]) if 0.35 < vals < 0.9999]
      add_values(story_dict, text1[i],[text2[i] for i in tryis], title)
      tf = 1
    else:
      tf = 0

    # For high structure similarity checking
    if any(vals >= 0.9999 for vals in token_cos[i]):
      tryis = [index for index, vals in enumerate(token_cos[i]) if vals >= 0.9999]
      add_values(story_dict, text1[i],[text2[i] for i in tryis], title)
      cp+=1
    elif any(0.7 < vals < 0.9999 for vals in token_cos[i]):
      tryis = [index for index, vals in enumerate(token_cos[i]) if 0.7 < vals < 0.9999]
      add_values(story_dict, text1[i],[text2[i] for i in tryis], title)
      token = 1
    else:
      token = 0

    # For plagiarism checking score (other than copy-pasted or 99% similarity)
    if token + tf > 1:
      plag_tf_token += 1
  return plag_tf_token, di, hs, cp

In [134]:
# Main code of the system, calling all the neccesary functions
def main_code(story_data,text_list):
  # Flattened the text to smooth out the tokenize and vectorize
  flat_text = [line for text in text_list for line in text]

  # Calling the tokenizer and vectorizer function
  tfid = vecTfid(flat_text,text_list)
  token = tokenizing(flat_text,text_list)

  plag_score = 0
  fin_plag_score = 0
  verdict = ""
  YN = 0
  story_dict = {}

  # Iterating each text to examine the similarity between text
  for j in range(len(token)):
    # Skip chapter text of same story
    if story_data.iloc[-1,1] != story_data.iloc[j,1]:
      plag_ft, di, hs, cp = count_flag(token[-1], token[j], tfid[-1], tfid[j], story_dict, text_list[-1], text_list[j], story_data.iloc[j,1:3])
      plag_score = (plag_ft+(hs+cp)/2)/di
    if plag_score > fin_plag_score:
      fin_plag_score = plag_score

  if fin_plag_score >= 0.3:
    YN = 1
    verdict = "Unfortunately, your plagiarism score has exceeded the maximum percentage. Please revise and try again."
  else:
    YN = 0
    verdict = "Congratulations, your plagiarism score is within safe percentage! You may upload your work!"

  final = pd.DataFrame({'Final Plagiarism Score': [fin_plag_score*100], 'Y/N': YN, 'Verdict': verdict})

  if len(story_dict) == 0:
    add_values(story_dict, '-', '-', ['-', '-'])

  data = [{'Line': key, 'Fiction_id': fic_id, 'Chapter_id': chap_id, 'Similar Line': value}
        for key, file_dict in story_dict.items()
        for fic_id, chap_dict in file_dict.items()
        for chap_id, value in chap_dict.items()]

  df = pd.DataFrame(data)
  df_e = df.explode('Similar Line')
  df_e.reset_index(drop=True, inplace=True)

  return final.to_json(), df_e.to_json()

In [126]:
def Plagiarism_Checker(story_data):
  story_url='https://readscape.live/pdftodatabase'
  text = text_list(story_data)
  show_arr = main_code(story_data,text)
  response = requests.post(story_url, data=[show_arr])
  if response.status_code == 200:
    print(response.json)
  return show_arr

In [43]:
def request(story_url='https://readscape.live/pdftodatabase'):
  story = requests.get(story_url)
  story = story.json()
  story = pd.DataFrame(story['data'])
  return story

In [135]:
final, df_e= Plagiarism_Checker(request())
df_e

<bound method Response.json of <Response [200]>>


'{"Line":{"0":"-"},"Fiction_id":{"0":"-"},"Chapter_id":{"0":"-"},"Similar Line":{"0":"-"}}'

In [136]:
final

'{"Final Plagiarism Score":{"0":0},"Y\\/N":{"0":0},"Verdict":{"0":"Congratulations, your plagiarism score is within safe percentage! You may upload your work!"}}'

In [139]:
df = pd.read_json(final)
df

,Final Plagiarism Score,Y/N,Verdict
0,0,0,"Congratulations, your plagiarism score is with..."
